In [2]:
%run PVModel_functions.ipynb

In [3]:
#This cell imports all the necessary packages & module data
import pandas as pd 
import numpy as np
from numpy import median
from datetime import datetime
import datatools
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import datetime
import pvlib
import pvanalytics
import math
import pvpltools
import seaborn as sns
import sklearn
import statistics
from sklearn.metrics import r2_score
from pvpltools import iec61853
from pvpltools.module_efficiency import adr, heydenreich, motherpv, pvgis, mpm5, mpm6, bilinear
from pvpltools.module_efficiency import fit_efficiency_model, fit_bilinear

%matplotlib inline 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

mpl.rcParams['font.size']=12
mpl.rcParams['lines.linewidth']=1
mpl.rcParams['xtick.labelsize']=10
#mpl.rcParams['font.weight']='bold'
mpl.rcParams['axes.titlesize']=22
sns.set(rc={'figure.figsize':(12,7)})

In [100]:
#This is where all the input info should be
#Valid module names : LG, Panasonic, Can270, Can275, HanQPlus, HanQPeak, MissionSolar, Jinko
mod_name = 'Itek'
met_name =  'dbo.SNLA_MET_DataOut'

In [101]:
mod_name

'Itek'

In [102]:
#Reads in an excel file of all necessary module coefficients, string lengths, system dates, etc
mod_info = pd.read_excel('module_coefficients.xlsx')
mod_info.index = mod_info['Manufacturer']
del mod_info['Manufacturer']
#defines the specific module given the previous user input and module coefficients file
module = mod_info[mod_name]
sys_name = module['table_name']
#start = '2020-01-01 00:00:00-07:00'
start = module['start']
#end = '2021-01-01 00:00:00-07:00'
end = module['end']

In [105]:
df.columns

Index(['RecNum_x', 'Global_Wm2_Avg', 'Direct_Wm2_Avg', 'Diffuse_Wm2_Avg',
       'Pressure_mBar_Avg', 'WS_ms_Mean', 'Wdir_Mean', 'Wdir_Std', 'WS_ms_Std',
       'WS_ms_3sec_Max', 'WD_deg_SMM', 'Temp_C_Avg', 'RH_pct_Avg',
       'Panel_Temp_C_Avg', 'Battery_V_Avg', 'ProcessTime_S_Max',
       'Global_mV_Avg', 'Rain_mm_Tot', 'Rain_mm_Daily', 'Temp_CMP22_C_Avg',
       'RecNum_y', 'BroadbandPOA_Avg', 'POACleanRC_E_Avg', 'GHI_Avg',
       'Albedo_Avg', 'AmbientTemp_Avg', 'PH1_V_Avg', 'PH2_V_Avg', 'PH3_V_Avg',
       'PH4_V_Avg', 'LM1_V_Avg', 'LM2_V_Avg', 'LM3_V_Avg', 'LM4_V_Avg',
       'PH1_I_Avg', 'PH2_I_Avg', 'PH3_I_Avg', 'PH4_I_Avg', 'LM1_I_Avg',
       'LM2_I_Avg', 'LM3_I_Avg', 'LM4_I_Avg', 'PH1_RTD_Avg', 'PH2_RTD_Avg',
       'PH3_RTD_Avg', 'PH4_RTD_Avg', 'LM1_RTD_Avg', 'LM2_RTD_Avg',
       'LM3_RTD_Avg', 'LM4_RTD_Avg', 'Wind_Speed_Avg', 'Wind_Direction_Avg',
       'Wind_Dir_std', 'Wind_Gust_Max', 'tcell', 'eff_irr', 'cell temp',
       'ratio str_1_i to eff_irr', 'ratio str_2_i to

In [103]:
#Import sys & met databases & merge into one df
df = sys_met_data(mettablename=met_name, systablename=sys_name, start=start, end=end)

OperationalError: (pymssql.OperationalError) (20009, b'DB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (DB03SNLNT\\PR)\n')
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [8]:
df.columns

Index(['RecNum_x', 'Global_Wm2_Avg', 'Direct_Wm2_Avg', 'Diffuse_Wm2_Avg',
       'Pressure_mBar_Avg', 'WS_ms_Mean', 'Wdir_Mean', 'Wdir_Std', 'WS_ms_Std',
       'WS_ms_3sec_Max', 'WD_deg_SMM', 'Temp_C_Avg', 'RH_pct_Avg',
       'Panel_Temp_C_Avg', 'Battery_V_Avg', 'ProcessTime_S_Max',
       'Global_mV_Avg', 'Rain_mm_Tot', 'Rain_mm_Daily', 'Temp_CMP22_C_Avg',
       'RecNum_y', 'Itek1_V_Avg', 'Itek2_V_Avg', 'Itek3_V_Avg', 'Itek4_V_Avg',
       'Itek5_V_Avg', 'Prism368_V_Avg', 'Prism375_V_Avg',
       'SolarWorld285_V_Avg', 'Itek1_I_Avg', 'Itek2_I_Avg', 'Itek3_I_Avg',
       'Itek4_I_Avg', 'Itek5_I_Avg', 'Prism368_I_Avg', 'Prism375_I_Avg',
       'SolarWorld285_I_Avg'],
      dtype='object')

In [22]:
#Convert measured pressure to pascal's to use in absolute solar/environmental conditions calculations (1mBar=100Pa)
pres = df['Pressure_mBar_Avg']*100

#Calculate Solar Data i.e. solar position, azimuth, zenith, etc.
sdf = calc_sol_data(time=df.index, latitude=module['latitude'], longitude=module['longitude'],
                    temperature=df['Temp_C_Avg'], tilt=module['tilt'], altitude=module['altitude'])

#Calculate Environmental Data i.e. airmass, pressure, POA irradiance compenents etc.
edf = calc_env_data(tilt=module['tilt'],altitude=module['altitude'], sur_azimuth=module['surface_azimuth'], 
                    sol_azimuth=sdf['azimuth'], zenith=sdf['apparent_zenith'], DNI=df['Direct_Wm2_Avg'], 
                    GHI=df['Global_Wm2_Avg'], DHI=df['Diffuse_Wm2_Avg'], DNI_extra=sdf['dni_extra'], model ='haydavies')

#Calculate airmass and add this to the Environmental Data
edf['am_abs'] = pvlib.atmosphere.get_absolute_airmass(airmass_relative=edf['airmass'], pressure=pres)

In [23]:
#Mask Solar Elevation below 15 deg and above 90 deg
sol_elev_mask = simple_filter(sys=sdf, variable='apparent_elevation', lower=15, upper=90, apply=False)
df = df.where(sol_elev_mask, np.nan)

In [24]:
#Calculate effective irradiance and cell temperature models using
sapm_data = sapm_param(POA_global=edf['poa_global'], POA_direct=edf['poa_direct'], POA_diffuse=edf['poa_diffuse'], 
                       amb_temp=df.Temp_C_Avg, ws_avg=df.WS_ms_Mean, am_abs=edf['am_abs'], aoi=edf['aoi'], module=module)
df = df.merge(sapm_data, how='inner', left_index=True, right_index=True)
df = df[~df.index.duplicated(keep='first')]

In [29]:
#Apply physical Filters on effective irradiance and cell temperature
df = simple_filter(sys=df, variable='eff_irr', lower=50, upper=1200, apply=True)
df = simple_filter(sys=df,variable='tcell', lower=-40, upper=85, apply=True)
df = simple_filter(sys=df, variable='Temp_C_Avg', lower=-10, upper=40, apply=True)

In [36]:
##### Filter out the data points with a certain ratio of current to irradiance
string_is ='str_1_i', 'str_2_i', 'str_3_i', 'str_4_i'
ratio_names = 'ratio str_1_i to eff_irr', 'ratio str_2_i to eff_irr', 'ratio str_3_i to eff_irr', 'ratio str_4_i to eff_irr'


for string_i in string_is:
    #Calculate the ratio at each given timestep
    df['ratio' + ' ' + string_i + ' ' 'to eff_irr'] = df[module[string_i]]/df['eff_irr']
    #Create a df with no NaNs to calculate the slope of trendline between current and irradiance
    nadf = df.dropna(inplace=False)
    z = np.polyfit(nadf['eff_irr'], nadf[module[string_i]],1)
    #Calculate lower/upper bounds
    lower = z[0]-(z[0]*0.3)
    upper = (z[0]*0.3) + z[0]
    #Use previously defined column name to filter values
    ratio_name ='ratio' + ' ' + string_i + ' ' 'to eff_irr'
    df = simple_filter(df, ratio_name, lower=lower, upper=upper, apply=True)  
    
#This loop will cycle through all 4 strings since performance may vary slightly for each string    

In [80]:
tmp = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']
df['tmod_avg'] = (df[module['str_1_rtd']]+df[module['str_2_rtd']]+df[module['str_3_rtd']]+df[module['str_4_rtd']])/4
df['cell temp'] = df['tmod_avg'] + ((df['eff_irr']/1000)*tmp['deltaT'])

temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']
tmod_sapm = pvlib.temperature.sapm_module(poa_global=edf['poa_global'], temp_air=df.Temp_C_Avg, 
                wind_speed=df.WS_ms_Mean,a=temperature_model_parameters['a'], b=temperature_model_parameters['b'])
df['tmod_sapm'] = tmod_sapm[~tmod_sapm.index.duplicated(keep='first')]                    

In [37]:
#Removing individual days that cause large errors due to shading, snow, etc

#filter out pole shading 
shade_list = ['Can270', 'HanQPlus']
if mod_name in shade_list:
    shade_start = '2020-10-06 00:00:00-07:00'
    shade_end = '2020-11-02 00:00:00-07:00'
    shask = (df.index > shade_start) & (df.index <= shade_end)
    df = df.loc[~shask]
    
#Filtering out days in 2019 that caused large outliers - could be due to snow on modules    
dec_list = ['Panasonic', 'LG']
if mod_name in dec_list:
    dec_start = '2019-12-29 00:00:00-07:00'
    dec_end = '2019-12-30 00:0:00-07:00'
    dec_mask = (df.index > dec_start) & (df.index <= dec_end)
    df = df.loc[~dec_mask]
    
#remove feb 23 2019
feb_start = '2019-02-23 00:00:00-07:00'
feb_end = '2019-02-24 10:00:00-07:00'
feb_mask = (df.index > feb_start) & (df.index <= feb_end)
df = df.loc[~feb_mask] 

#remove dec 2 2018
dec_start = '2018-12-02 00:00:00-07:00'
dec_end = '2018-12-02 23:59:00-07:00'
dec_mask = (df.index > dec_start) & (df.index <= dec_end)
df = df.loc[~dec_mask]

#remove dec 26 2018
dec_start = '2018-12-26 00:00:00-07:00'
dec_end = '2018-12-26 23:59:00-07:00'
dec_mask = (df.index > dec_start) & (df.index <= dec_end)
df = df.loc[~dec_mask]

#Removing Days in 2020 that had extremely large differences 
#remove sep 21 2020
sep_start = '2020-09-21 00:00:00-07:00'
sep_end = '2020-09-22 23:00:00-07:00'
sep_mask = (df.index > sep_start) & (df.index <= sep_end)
df = df.loc[~sep_mask]

#remove feb 5 2020
feb_start = '2020-02-05 00:00:00-07:00'
feb_end = '2020-02-05 23:00:00-07:00'
feb_mask = (df.index > feb_start) & (df.index <= feb_end)
df = df.loc[~feb_mask]

In [38]:
#Generate a results dataframe and add measured values
str_1 = meas_val(str_v=df[module['str_1_v']],str_i=df[module['str_1_i']])
str_2 = meas_val(str_v=df[module['str_2_v']],str_i=df[module['str_2_i']])
str_3 = meas_val(str_v=df[module['str_3_v']],str_i=df[module['str_3_i']])
str_4 = meas_val(str_v=df[module['str_4_v']],str_i=df[module['str_4_i']])

In [82]:
#Generate empty df to store appended results
newresults=[]

In [83]:
#Calcualte SAPM results and add to df
dc = sapm(eff_irr=df['eff_irr'], tcell=df['tcell'] , module=module)
sapm_results = sapm_p(v_mp=dc['v_mp'], i_mp=dc['i_mp'], str_len=module['str_len']).to_frame()


sapm_results.rename(columns = {0 : 'Modeled Power'}, inplace = True)
sapm_results['Str_1'] = str_1
sapm_results['Str_2'] = str_2
sapm_results['Str_3'] = str_3
sapm_results['Str_4'] = str_4
sapm_results['Time'] = sapm_results.index
sapm_results['Year'] = sapm_results.index.year
sapm_results['Model Name'] = 'SAPM'
newresults.append(sapm_results)

In [84]:
#Calculate PVWatts results and add to df
pvw_results = (pvwatts(POA_eff = df['eff_irr'], cell_temp = df['tcell'] , stc_mod_p = module['Power'], 
                Gpmp = (module['gamma_pmp']/100),str_len = module['str_len'],temp_ref=25.0 )).to_frame()

pvw_results.rename(columns = {0 : 'Modeled Power'}, inplace = True)
pvw_results['Str_1'] = str_1
pvw_results['Str_2'] = str_2
pvw_results['Str_3'] = str_3
pvw_results['Str_4'] = str_4
pvw_results['Time'] = pvw_results.index
pvw_results['Year'] = pvw_results.index.year
pvw_results['Model Name'] = 'PVW'
newresults.append(pvw_results)

In [85]:
#Calculate parameters to use in single diode models
cec_ivt = pvlib.ivtools.sdm.fit_cec_sam(celltype=module['cell_type'], v_mp=module['Vmpp'], i_mp=module['Impp'],
                                v_oc=module['Voc'], i_sc=module['Isc'],alpha_sc=module['Alisc'], 
                                beta_voc=module['Bvoc'],gamma_pmp=(module['Gampmp']),
                                cells_in_series=module['Cells_in_Series'])

#Calculate parameters necessary to input into single diode model
cec_param = pvlib.pvsystem.calcparams_cec(effective_irradiance=df['eff_irr'], 
            temp_cell=df['tcell'] , alpha_sc=module['Alisc'],a_ref=cec_ivt[4], I_L_ref=cec_ivt[0], 
            I_o_ref=cec_ivt[1], R_sh_ref=cec_ivt[3],R_s=cec_ivt[2],Adjust=cec_ivt[5],EgRef=1.121)
#Calculate power using single diode model
cec_p = pvlib.pvsystem.singlediode(photocurrent = cec_param[0], saturation_current = cec_param[1], resistance_series =
                           cec_param[2], resistance_shunt = cec_param[3], nNsVth = cec_param[4], method = 'newton')
#Calculate string power
cec_results = sdm(p_mp=cec_p['p_mp'], str_len=module['str_len']).to_frame()


cec_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
cec_results['Str_1'] = str_1
cec_results['Str_2'] = str_2
cec_results['Str_3'] = str_3
cec_results['Str_4'] = str_4
cec_results['Time'] = cec_results.index
cec_results['Year'] = cec_results.index.year
cec_results['Model Name'] = 'CEC'
newresults.append(cec_results)

In [86]:
#Calculate Desoto results and add to df

#Calculate parameters necessary to input into single diode model
ds_param = pvlib.pvsystem.calcparams_desoto(effective_irradiance=df['eff_irr'], 
            temp_cell=df['tcell'], alpha_sc=module['Alisc'],a_ref=cec_ivt[4], I_L_ref=cec_ivt[0], 
            I_o_ref=cec_ivt[1], R_sh_ref=cec_ivt[3],R_s=cec_ivt[2], EgRef=1.121)
#Calculate power using single diode model
ds_p = pvlib.pvsystem.singlediode(photocurrent = ds_param[0], saturation_current = ds_param[1], resistance_series =
                           ds_param[2], resistance_shunt = ds_param[3], nNsVth = ds_param[4], method = 'newton')
#Calculate string power
des_results = sdm(p_mp=ds_p['p_mp'], str_len=module['str_len']).to_frame()

des_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
des_results['Str_1'] = str_1
des_results['Str_2'] = str_2
des_results['Str_3'] = str_3
des_results['Str_4'] = str_4
des_results['Time'] = des_results.index
des_results['Year'] = des_results.index.year
des_results['Model Name'] = 'DES'
newresults.append(des_results)

In [87]:
Io_cfv = ((module['i_sc']+((module['i_sc']*module['R_s'])/module['R_sh_ref']))-(module['v_oc']/module['R_sh_ref']))/\
            (np.exp((module['v_oc']*(1.60217662e-19))/(module['gamma_ref']*module['Cells_in_Series']*(1.38064852e-23)*(298.15)))-1)

In [88]:
#Calculte PVSystem results and add to df
I_L = module['i_sc'] + ((module['i_sc']*module['R_s'])/module['R_sh_ref'])
#Calculate I_o_ref since this is the only value missing from the PAN files needed to calculate the PVS Parameters
Io = -((module['i_mp']+(module['v_mp'] + module['i_mp']*module['R_s'])/module['R_sh_ref'])-module['i_sc'])/\
            (np.exp((1.60217662e-19)*(module['v_mp']+module['i_mp']*module['R_s'])/\
            (module['Cells_in_Series']*module['gamma_ref']*(1.38064852e-23)*(298.15)))-1)
#Calculate parameters necessary to input into single diode model
pvs_param = pvlib.pvsystem.calcparams_pvsyst(effective_irradiance=df['eff_irr'],temp_cell=df['tcell'] , 
            alpha_sc=module['alpha_sc'],gamma_ref=module['gamma_ref'],mu_gamma=module['mu_gamma'], 
            I_L_ref=I_L,I_o_ref=Io_cfv,R_sh_ref=module['R_sh_ref'], R_sh_0=module['R_sh_0'],
            R_s=module['R_s'],cells_in_series=module['Cells_in_Series'], EgRef=1.121)
#Calculate power using single diode model
pvs_p = pvlib.pvsystem.singlediode(photocurrent = pvs_param[0], saturation_current = pvs_param[1], resistance_series =
                           pvs_param[2], resistance_shunt = pvs_param[3], nNsVth = pvs_param[4], method = 'newton')
#Calculate string power
pvs_results = sdm(p_mp=pvs_p['p_mp'], str_len=module['str_len']).to_frame()

pvs_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
pvs_results['Str_1'] = str_1
pvs_results['Str_2'] = str_2
pvs_results['Str_3'] = str_3
pvs_results['Str_4'] = str_4
pvs_results['Time'] = pvs_results.index
pvs_results['Year'] = pvs_results.index.year
pvs_results['Model Name'] = 'PVS'
newresults.append(pvs_results)

In [89]:
#Import matrix of measurement data
measurements_file = 'Sandia_PV_Module_P-Matrix-and-TempCo-Data_2019.xlsx'
matrix = pd.read_excel(measurements_file, sheet_name = mod_name, usecols='B,C,H', header=None, skiprows=5, nrows=27)
matrix.columns = ['temperature', 'irradiance', 'p_mp']
# calculate efficiency from power
matrix = matrix.eval('eta = p_mp / irradiance')
eta_stc = matrix.query('irradiance == 1000 and temperature == 25').eta
matrix.eta /= eta_stc.values
# just keep the columns that are needed
matrix = matrix[['irradiance', 'temperature', 'eta']]

In [90]:
#Calculate Heydenreich results and add to df
#Determine the parameters for the eff. model using non-linear least squares fit
popt, pcov = fit_efficiency_model(irradiance=matrix.irradiance, temperature=matrix.temperature,
                                  eta=matrix.eta, model=adr)
adr_eff = adr(df['eff_irr'], df['tmod_sapm'], *popt)
adr_results = ((df['eff_irr']/1000) * (adr_eff) * (module['Power']) *12).to_frame()

adr_results.rename(columns = {'eff_irr' : 'Modeled Power'}, inplace = True)
adr_results['Str_1'] = str_1
adr_results['Str_2'] = str_2
adr_results['Str_3'] = str_3
adr_results['Str_4'] = str_4
adr_results['Time'] = adr_results.index
adr_results['Year'] = adr_results.index.year
adr_results['Model Name'] = 'ADR'
newresults.append(adr_results)

In [91]:
#Calculate Heydenreich results and add to df
#Determine the parameters for the eff. model using non-linear least squares fit
popt, pcov = fit_efficiency_model(irradiance=matrix.irradiance, temperature=matrix.temperature,
                                  eta=matrix.eta, model=heydenreich)
hey_eff = heydenreich(df['eff_irr'],df['tmod_sapm'], *popt)
hey_results = ((df['eff_irr']/1000) * (hey_eff) * (module['Power']) *12).to_frame()

hey_results.rename(columns = {'eff_irr' : 'Modeled Power'}, inplace = True)
hey_results['Str_1'] = str_1
hey_results['Str_2'] = str_2
hey_results['Str_3'] = str_3
hey_results['Str_4'] = str_4
hey_results['Time'] = hey_results.index
hey_results['Year'] = hey_results.index.year
hey_results['Model Name'] = 'HEY'
newresults.append(hey_results)

In [92]:
#Calculate MotherPV results and add to df
#Determine the parameters for the eff. model using non-linear least squares fit
popt, pcov = fit_efficiency_model(irradiance=matrix.irradiance,temperature=matrix.temperature,eta=matrix.eta,model=motherpv)
mother_eff = motherpv(df['eff_irr'], df['tmod_sapm'], *popt)
mot_results = ((df['eff_irr']/1000) * (mother_eff) * (module['Power']) * 12).to_frame()

mot_results.rename(columns = {'eff_irr' : 'Modeled Power'}, inplace = True)
mot_results['Str_1'] = str_1
mot_results['Str_2'] = str_2
mot_results['Str_3'] = str_3
mot_results['Str_4'] = str_4
mot_results['Time'] = mot_results.index
mot_results['Year'] = mot_results.index.year
mot_results['Model Name'] = 'MOT'
newresults.append(mot_results)

In [93]:
#Calculate PVGIS results and add to df
#Determine the parameters for the eff. model using non-linear least squares fit
popt, pcov = fit_efficiency_model(irradiance=matrix.irradiance,temperature=matrix.temperature,eta=matrix.eta,model=pvgis)
pvgis_eff = pvgis(df['eff_irr'], df['tmod_sapm'], *popt)
pvg_results = ((df['eff_irr']/1000) * (pvgis_eff) * (module['Power']) *12).to_frame()

pvg_results.rename(columns = {'eff_irr' : 'Modeled Power'}, inplace = True)
pvg_results['Str_1'] = str_1
pvg_results['Str_2'] = str_2
pvg_results['Str_3'] = str_3
pvg_results['Str_4'] = str_4
pvg_results['Time'] = pvg_results.index
pvg_results['Year'] = pvg_results.index.year
pvg_results['Model Name'] = 'PVG'
newresults.append(pvg_results)

In [94]:
#Calculate MPM5 results and add to df
#Determine the parameters for the eff. model using non-linear least squares fit
popt, pcov = fit_efficiency_model(irradiance=matrix.irradiance,temperature=matrix.temperature,eta=matrix.eta,model=mpm5)
mpm5_eff = mpm5(df['eff_irr'], df['tmod_sapm'], *popt)
mpm5_results = ((df['eff_irr']/1000) * (mpm5_eff) * (module['Power']) *12).to_frame()

mpm5_results.rename(columns = {'eff_irr' : 'Modeled Power'}, inplace = True)
mpm5_results['Str_1'] = str_1
mpm5_results['Str_2'] = str_2
mpm5_results['Str_3'] = str_3
mpm5_results['Str_4'] = str_4
mpm5_results['Time'] = mpm5_results.index
mpm5_results['Year'] = mpm5_results.index.year
mpm5_results['Model Name'] = 'MPM5'
newresults.append(mpm5_results)

In [95]:
#Calculate MPM6 results and add to df
#Determine the parameters for the eff. model using non-linear least squares fit
popt, pcov = fit_efficiency_model(irradiance=matrix.irradiance,temperature=matrix.temperature,eta=matrix.eta,model=mpm6)
mpm6_eff = mpm6(df['eff_irr'], df['tmod_sapm'], *popt)
mpm6_results = ((df['eff_irr']/1000) * (mpm6_eff) * (module['Power']) *12).to_frame()

mpm6_results.rename(columns = {'eff_irr' : 'Modeled Power'}, inplace = True)
mpm6_results['Str_1'] = str_1
mpm6_results['Str_2'] = str_2
mpm6_results['Str_3'] = str_3
mpm6_results['Str_4'] = str_4
mpm6_results['Time'] = mpm6_results.index
mpm6_results['Year'] = mpm6_results.index.year
mpm6_results['Model Name'] = 'MPM6'
newresults.append(mpm6_results)

In [96]:
#Calculate Bilinear results and add to df
#Determine the parameters for the eff. model using non-linear least squares fit
interpolator = fit_bilinear(**matrix)
bilinear_eff = bilinear(df['eff_irr'],df['tmod_sapm'], interpolator)
bil_results = ((df['eff_irr']/1000) * (bilinear_eff) * (module['Power']) * 12).to_frame()

bil_results.rename(columns = {'eff_irr' : 'Modeled Power'}, inplace = True)
bil_results['Str_1'] = str_1
bil_results['Str_2'] = str_2
bil_results['Str_3'] = str_3
bil_results['Str_4'] = str_4
bil_results['Time'] = bil_results.index
bil_results['Year'] = bil_results.index.year
bil_results['Model Name'] = 'BIL'
newresults.append(bil_results)

In [97]:
metasingle = pd.concat(newresults, ignore_index=True)
metasingle.index=metasingle['Time']

In [98]:
metasingle.to_hdf('metasingle2.h5', key='data', complib='zlib', complevel=5)